<a href="https://colab.research.google.com/github/kriaz100/deep-learning-with-python-notebooks/blob/master/chapter11_part02_sequence-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

### Processing words as a sequence: The sequence model approach

Instead of manually crafting order based features, we expose model to raw word sequences and let it figure out features on its own -- ie. leverage power of deep learning.

#### A first practical example
Let's try the first sequence model in parcatice. 



**Downloading the data**

IMDB data

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  27.0M      0  0:00:02  0:00:02 --:--:-- 27.0M


**Preparing the data**

Preparing the directory structure

Moving datsets into relevant directories using Keras utility `text_dataset_from_directory`

Dont forget to modify `os.mkdirs()` by adding `exist_ok=True`, in case the directory already exists


In [3]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category, exist_ok=True)  #adding 'exist_ok=True'
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Preparing integer sequence datasets**

First we prepare the datasets that return integer sequences.
We use textVectorizationlayer with output mode="int"


We also limit vocabulary size to 600 (ignoring less frequently occuring words) 

In [4]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequence**

- To convert integer sequences to vector sequence, we  one-hot-encoding `tf.one_hot()`

- Next we add bidirectional LSTM on top of vectors `layers.Bidirectional(layers.LSTM(32))`
- Dropout layer is also added `layers.Dropout()
- Finally add a calssification layer `sigmoid`

In [5]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)         # one-hot-encoding
x = layers.Bidirectional(layers.LSTM(32))(embedded)     # bi-directional LSTM
x = layers.Dropout(0.5)(x)                              # dropout layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

**Training a first basic sequence model**

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 454s 712ms/step - loss: 0.5288 - accuracy: 0.7516 - val_loss: 0.4018 - val_accuracy: 0.8468
Epoch 2/10
625/625 [==============================] - 442s 707ms/step - loss: 0.3401 - accuracy: 0.8787 - val_loss: 0.3067 - val_accuracy: 0.8834
Epoch 3/10
625/625 [==============================] - 443s 708ms/step - loss: 0.2698 - accuracy: 0.9069 - val_loss: 0.2960 - val_accuracy: 0.8832
Epoch 4/10
625/625 [==============================] - 444s 711ms/step - loss: 0.2331 - accuracy: 0.9183 - val_loss: 0.2868 - val_accuracy: 0.8864
Epoch 5/10
625/625 [==============================] - 444s 710ms/step - loss: 0.2033 - accuracy: 0.9307 - val_loss: 0.3046 - val_accuracy: 0.8862
Epoch 6/10
625/625 [==============================] - 445s 712ms/step - loss: 0.1757 - accuracy: 0.9402 - val_loss: 0.3255 - val_accuracy: 0.8616
Epoch 7/10
625/625 [==============================] - 445s 712ms/step - loss: 0.1653 - accuracy: 0.9446 - val_loss: 0.3068 -

Using one-hot encoding to convert words to vectors wsa not a great idea. There is a better way, *word imbeddings*

#### Understanding word embeddings
- One-hot encoding is a way of feature engineering. One-hot vectors are orthogonal to each other. The different tokens are assumed to be independent of each other.  

- But this is not true. Some words share information with each other. Words "movie" and "film" are interchangeable. They vectors cannot be orthogonal to each other. They should be the same vector or close enough (L2 distance or cosine distance between them should be less).

- We expect geometric distance between any to word vectors to be related to semantic distance between them -- vector for related words should lie close to each other; and for words that mean different things, these vector shold be far away from each other.

<font color='blue'>***Word embeddings*** are vector representations of words that achieve exactly this: they map human language into a structured geometric space</font>

Word embedding high dimensional vectors that are dense (floating-point vectors), with lower dimensionality (256-dimensional, 512-dimensional, or 1024-dimensional). This contrasts with one-hot vectors that are sparse (mostly zeros) and high dimensions (e.g 20,000) are need to capture large (say 20,000 word) vocabulary. So word embeddings pack more information due to denseness.

<font color='blue'>*The word embeddings are structural representations. Their structure is learned from data*</font>

- Similar words get embedded in close locations. 
- specific directons in the embedding space are meaningful.

**There are two ways to obtain word embeddings**

  1.  Learn them from data, jointly with the main task such as document classification or sentiment prediction.

  2.  Load them into your model word embeddings precomputed (learned) from a different machine learning task than the one you are trying to solve. These are called <font color='blue'>pre-trained word embeddings</font>.

#### Learning word embeddings with the Embedding layer


**Instantiating an `Embedding` layer**

In [ ]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

**Model that uses an `Embedding` layer trained from scratch**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

#### Understanding padding and masking

**Using an `Embedding` layer with masking enabled**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

#### Using pretrained word embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

**Parsing the GloVe word-embeddings file**

In [ ]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

**Preparing the GloVe word-embeddings matrix**

In [ ]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

**Model that uses a pretrained Embedding layer**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")